In [75]:
import igraph as ig
import numpy as np
from copy import deepcopy
from typing import Callable, List
import plotly.express as px
import math
import pandas as pd


In [68]:
g = ig.Graph.Read('../../data/graphs/april2022_Lspace.graphml')


In [69]:
def opercolation(graph: ig.Graph, centrality_measure: Callable[[ig.Graph], List[float]]):
    tmp_g = deepcopy(graph)  # Make a copy of the graph
    # Result list containing tuples (n of removed nodes, size of the biggest component)
    remnodes_bcsize = []
    one_percent_size = int(math.ceil(len(tmp_g.vs) / 100))
    perc_removed = 0

    while len(tmp_g.vs) >= one_percent_size:

        # Biggest component size
        bc_size = max(len(c) for c in tmp_g.components(mode='strong'))
        S = (bc_size / len(g.vs)) * 100
        remnodes_bcsize.append((S, perc_removed))
        perc_removed += 1

        # Calculate centrality measure
        centrality = np.array(centrality_measure(tmp_g))
        max_centrality_vertex_ids = np.argpartition(
            centrality, -one_percent_size)[-one_percent_size:]

        # Delete vertex with max centrality measure
        tmp_g.delete_vertices(max_centrality_vertex_ids)
    return remnodes_bcsize


In [78]:
def plot_opercolation(centrality_measure_names, *args):
    S = np.concatenate(
        [np.array([S for S, _ in x]) for x in args],
        axis=None
    )

    rem_perc = np.concatenate(
        [np.array([p for _, p in x]) for x in args],
        axis=None
    )

    centrality_measure = np.concatenate(
        [np.array([centrality_measure_names[i] for _ in x]) for i, x in enumerate(args)],
        axis=None
    )

    data = pd.DataFrame(dict(
        S=S,
        rem_perc=rem_perc,
        centrality_measure=centrality_measure
    ))  

    fig = px.line(data, x="rem_perc", y="S", color="centrality_measure", line_dash="centrality_measure")
    fig.show()

In [71]:
degree_op = opercolation(g, lambda x: x.degree())
strength_op = opercolation(g, lambda x: x.strength(weights='num_train'))
betweennes_op = opercolation(g, lambda x: x.betweenness())
closeness_op = opercolation(g, lambda x: x.closeness()) 
pagerank_op = opercolation(g, lambda x: x.pagerank())
pagerank_numtrain_op = opercolation(g, lambda x: x.pagerank(weights='num_train'))


In [79]:
plot_opercolation(
    ['degree', 'betweennes', 'closeness', 'pagerank', 'strength_num_train', 'pagerank_num_train'], 
    degree_op, betweennes_op, closeness_op, pagerank_op, strength_op, pagerank_numtrain_op
)